### Convert to Binary Image:
Use image processing software (e.g., ImageJ, Python, MATLAB) to threshold the image and create a binary mask where pores are white (1) and the polymer film is black (0).

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure

image_path = '0.1%B.png'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Preprocess the image (thresholding to highlight pores)
_, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV)

# Find contours of the pores
contours = measure.find_contours(binary_image, 0.8)

# Extract centroids of the pores
centroids = []
for contour in contours:
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])  # X-coordinate of centroid
        cy = int(M["m01"] / M["m00"])  # Y-coordinate of centroid
        centroids.append((cx, cy))

centroids = np.array(centroids)

In [5]:
from scipy.spatial import Voronoi, voronoi_plot_2d

# Compute the Voronoi diagram
vor = Voronoi(centroids)

# Plot the Voronoi diagram
fig, ax = plt.subplots(figsize=(10, 10))
voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors='blue', line_width=1, line_alpha=0.6)

# Overlay the pore centroids
ax.scatter(centroids[:, 0], centroids[:, 1], c='red', s=10, marker='o', label='Pore Centers')
ax.set_title("Voronoi Analysis of Breath Figure Array")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.legend()
plt.show()

QhullError: QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull v Qc Qz Qbb
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1579129256  voronoi  Qcoplanar-keep  Qz-infinity-point  Qbbound-last
  _pre-merge  _zero-centrum  Qinterior-keep  Pgood  _max-width 1.2e+03
  Error-roundoff 1.7e-12  _one-merge 1.2e-11  Visible-distance 3.5e-12
  U-max-coplanar 3.5e-12  Width-outside 6.9e-12  _wide-facet 2.1e-11
  _maxoutside 1.4e-11
